## importing required packages

In [85]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import re
import pickle
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## data processing

In [10]:
# loading data from csv to pandas dataframe
tweets = pd.read_csv('/dataset/sentimentTwitter.csv', encoding='ISO-8859-1')
tweets.shape

(1599999, 6)

In [11]:
tweets.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [12]:
# reloading data with column names

cols = ['target', 'id', 'date', 'flag', 'user', 'tweet']
tweets = pd.read_csv('/content/sentimentTwitter.csv',names=cols, encoding='ISO-8859-1')
tweets.shape

(1600000, 6)

In [14]:
tweets.head()

,target,id,date,flag,user,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [16]:
# counting missing values
tweets.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
tweet     0
dtype: int64

In [17]:
# checking target column
tweets['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [20]:
# convert 4 to 1
tweets.replace({'target':{4:1}}, inplace=True)
tweets['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 -> -ve tweet<br>
1 -> +ve tweet

## stemming

In [28]:
engStopwords = stopwords.words('english')
pattern = re.compile('[^a-zA-Z]')
porterStemmer = PorterStemmer()
def stemming(text):
    newText = re.sub(pattern, ' ', text)
    newText = newText.lower()
    newText = newText.split()
    newText = [porterStemmer.stem(word) for word in newText if not word in engStopwords]
    newText = ' '.join(newText)
    return newText

In [29]:
tweets['stemmedTweets'] = tweets['tweet'].apply(stemming)

In [30]:
tweets.head()

,target,id,date,flag,user,tweet,stemmedTweets
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [33]:
tweets['stemmedTweets']

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmedTweets, Length: 1600000, dtype: object

In [34]:
tweets['target']

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64

In [39]:
# separating data and label
x = tweets['stemmedTweets'].values
y = tweets['target'].values

In [40]:
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [41]:
print(y)

[0 0 0 ... 1 1 1]


## vector space model for text


In [75]:
# spliting data to training and testing data

xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.15, stratify=y, random_state=5)

In [76]:
print(xTrain)

['roundgoldfish cri'
 'gener plea market depart somebodi give someon advertis brief kid look malnourish atm'
 'littl hung good' ...
 'catatoniccat keep hair check roof look like got fro wear hat'
 'work ruin life miss everyth work'
 'still work half compani ballgam other dinner']


In [77]:
print(xTest)

['friday alreadi' 'laurenrevengexo omg total agre fuck miley ili paramor'
 'caught smile hot hairdress girl morn life good' ...
 'mand offend comment sound like tweeti bird talk'
 'still brochur work come ear thought design suppos fun work eat shirt design time'
 'hadtobey thanker miss andi phew awesom']


In [78]:
# converting text data to numerical data

vectorizer = TfidfVectorizer()
xTrain = vectorizer.fit_transform(xTrain)
xTest = vectorizer.transform(xTest)

In [79]:
print(xTrain)

  (0, 87689)	0.4285046628196861
  (0, 359735)	0.9035395696602264
  (1, 27927)	0.2641163640006205
  (1, 259854)	0.4348029606309523
  (1, 250182)	0.14945741717225489
  (1, 225133)	0.1943332026482168
  (1, 54221)	0.31884153896815604
  (1, 5102)	0.29335432426764985
  (1, 390371)	0.18757500295223184
  (1, 154202)	0.18808921817342994
  (1, 390292)	0.2564469573016374
  (1, 101781)	0.30943274157854783
  (1, 263048)	0.24929356617246975
  (1, 331222)	0.3670642029022141
  (1, 150758)	0.2537918863425167
  (2, 156721)	0.34814715674263796
  (2, 178726)	0.7944415435405372
  (2, 246737)	0.49765067180598255
  (3, 176997)	0.23463947802825835
  (3, 86729)	0.422822742518079
  (3, 13028)	0.26436443420827166
  (3, 453950)	0.20263734959903806
  (3, 142034)	0.2799054507174099
  (3, 148174)	0.2584632528038147
  (3, 389342)	0.3679255768320117
  :	:
  (1359996, 285629)	0.27770376933779856
  (1359996, 156721)	0.22390605468315813
  (1359997, 65638)	0.4915056019960569
  (1359997, 144677)	0.4140202619189412
  (13599

In [80]:
print(xTest)

  (0, 144282)	0.7359071154344479
  (0, 12914)	0.6770825041698759
  (1, 430168)	0.3273111111207423
  (1, 320929)	0.480937886177269
  (1, 313357)	0.30600252618790874
  (1, 276586)	0.3843420415061147
  (1, 184065)	0.433403775199344
  (1, 145465)	0.319538477145017
  (1, 6551)	0.36092404261918526
  (2, 387665)	0.3699072009204075
  (2, 285629)	0.25761182657283194
  (2, 243552)	0.29748734687836315
  (2, 176655)	0.3209869245256346
  (2, 164121)	0.5453210306184565
  (2, 156721)	0.20770639111304912
  (2, 153754)	0.29893325765962203
  (2, 66258)	0.41880236194284864
  (3, 463322)	0.3195083504269405
  (3, 397071)	0.4293719612548018
  (3, 347960)	0.4653126205953711
  (3, 161637)	0.4764320138638856
  (3, 151698)	0.3002453629517968
  (3, 41609)	0.4241503933770127
  (4, 462338)	0.34172186682418176
  (4, 423552)	0.3285926559107748
  :	:
  (239997, 436081)	0.36710960448697644
  (239997, 409772)	0.2487632229862481
  (239997, 392074)	0.2551463574470774
  (239997, 310603)	0.42269231834826343
  (239997, 2603

## training the ml model - logistic regression

In [81]:
model = LogisticRegression(max_iter=1000)

In [82]:
model.fit(xTrain, yTrain)

LogisticRegression(max_iter=1000)

## model accuracy score

In [83]:
# accuracy score on training data
xTrainPredict = model.predict(xTrain)
traingAccuracy = accuracy_score(yTrain, xTrainPredict)
traingAccuracy

0.8102419117647058

In [84]:
# accuracy score on testing data
xTestPredict = model.predict(xTest)
testgAccuracy = accuracy_score(yTest, xTestPredict)
testgAccuracy

0.7774916666666667

## saving trained model as pickle file

In [86]:
pickle.dump(model, open('twitterModel.pkl', 'wb'))